In [1]:
from pprint import pprint

import mysql.connector
from containers import Paper, Author, Expertise, Institution, JournalConference

HOST    = "localhost"
USER    = "root"
PASSWD  = "0715Phj!"
DB_USE  = "paper_clip"
DB_USE  = "relation_db_project"

import os
import json
import platform

from containers import QueryHandler

os_name = platform.system()

WHOLE_PAPER_FILE_PATH = "./data/processed_paper_dict.json"
WHOLE_PAPER_FILE_PATH = "./data/final_paper_dict.json"
if os.path.exists(WHOLE_PAPER_FILE_PATH) :
    if os_name == "Windows" :
        with open(WHOLE_PAPER_FILE_PATH, "r", encoding="utf-8") as f :
            whole_paper_dict = json.load(f)
    else :
        with open(WHOLE_PAPER_FILE_PATH, "r") as f :
            whole_paper_dict = json.load(f)
    for k, v in whole_paper_dict.items() :
        whole_paper_dict[k] = Paper(**v)

class QueryHandler_ :
    def __init__(
        self,
        host    = None,
        user    = None,
        passwd  = None,
        db_use  = None
    ) :
        self.mydb = mysql.connector.connect(
            host    = host,
            user    = user,
            passwd  = passwd,
            database= db_use
        )
        self.cursor = self.mydb.cursor(prepared=True)

    def paperByDOI(self, doi) :
        paper_list = self.queryPaperBy("DOI", doi)
        if len(paper_list) == 0 :
            paper = Paper(DOI=doi, title=doi)
            return False, paper
        else :
            return True, paper_list[0]

    def queryPaperBy(self, by, value) :
        self.cursor.execute(f"""
            select p.DOI, p.title,  p.referenced_num, GROUP_CONCAT(' ', apr.author_name) as author_name, p.keywords, c.`name` as conference_name, p.clip
            from paper p
            Left join author_paper_relationship apr on p.DOI = apr.DOI
            left join conference c on p.issn = c.issn
            where {by} like '%{value}%'
            GROUP BY p.DOI, p.title,  p.referenced_num, p.keywords, c.`name`
            order by p.referenced_num desc
            limit 100;
        """)
        paper_dict = {}
        for row in self.cursor :
            doi, title, refernced_num, author_name, keywords, conference_name, is_in_favorite = row
            if doi not in paper_dict :
                paper_dict[doi] = Paper(
                    DOI             = doi,
                    title           = title,
                    authors         = list(map(
                        lambda x : x.strip(),
                        author_name.split(', ')
                    )) if author_name is not None else None, #[''],
                    keywords        = list(map(
                        lambda x : x.strip(),
                        keywords.split(', ')
                    )) if keywords is not None else None, # [''],
                    conference_acronym = conference_name,
                    referenced_num  = refernced_num,
                    reference_list = [],
                    is_in_favorite  = is_in_favorite,
                    query_handler=self
                )
        return list(paper_dict.values())

    def fillReferenceList(self, paper) :
        self.cursor.execute(f"""
            select ref_doi
            from referenced_paper
            where DOI like '%{paper.DOI}%'
            ;
        """)
        for row in self.cursor :
            paper.reference_list.append(row[0])

    def queryPaperBy_(self, by, value, exact=True) :
        
        self.cursor.execute(f"""
            SELECT
                p.DOI,
                p.title AS paper_title,
            GROUP_CONCAT(' ', apr.author_name) AS author_names,
                p.keywords,
                c.`name` AS conference_name,
                p.referenced_num,
                p.abstract,
                rp.ref_doi AS referenced_paper_doi,
                referenced_paper.title AS referenced_paper_title,
            GROUP_CONCAT(' ', referenced_author.author_name) AS referenced_paper_author_names
            FROM
                paper p
            LEFT JOIN
                author_paper_relationship apr ON p.DOI = apr.DOI
            LEFT JOIN
                conference c ON p.issn = c.issn
            LEFT JOIN
                referenced_paper rp ON p.DOI = rp.DOI
            LEFT JOIN
                paper AS referenced_paper ON rp.ref_doi = referenced_paper.DOI
            LEFT JOIN
                author_paper_relationship AS referenced_author ON referenced_paper.DOI = referenced_author.DOI
            WHERE
                #p.DOI LIKE '10.1109/tpami.2016.2577031%'
                {by} LIKE "%{value}%"
            GROUP BY
                p.DOI, paper_title, p.keywords, c.`name`, p.referenced_num, p.abstract, rp.ref_doi, referenced_paper_title
            ORDER BY
                p.referenced_num DESC
            ;
        """)
        paper_dict = {}

        for row in self.cursor :
            doi, title, authors, keywords, conference_name, refernced_num, abstract, ref_doi, ref_title, ref_authors = row
            if doi not in paper_dict :
                paper_dict[doi] = Paper(
                    DOI             = doi,
                    title           = title,
                    authors         = list(map(
                        lambda x : x.strip(),
                        authors.split(', ')
                    )) if authors is not None else None, #[''],
                    keywords        = list(map(
                        lambda x : x.strip(),
                        keywords.split(', ')
                    )) if keywords is not None else None, # [''],
                    conference_acronym = conference_name,
                    referenced_num  = refernced_num,
                    abstract        = abstract,
                    reference_list  = [ref_doi] if ref_doi is not None else [],
                    query_handler=self
                )
            else :
                paper_dict[doi].reference_list.append(ref_doi)

        return list(paper_dict.values())

    def updatePaper(self, paper) :
        # set clip column of paper table to paper.is_in_favorite
        print(paper.is_in_favorite)
        self.cursor.execute(f"""
            UPDATE paper
            SET clip = {paper.is_in_favorite}
            WHERE DOI LIKE "%{paper.DOI}%"
            ;
        """)
        self.mydb.commit()


In [2]:
query_handler = QueryHandler(
    host=HOST,
    user=USER,
    passwd=PASSWD,
    db_use=DB_USE
)

BY = "p.DOI"
BY = "p.keywords"

VALUE = "10.1109/tpami.2016.2577031"
VALUE = "deep learning"

VALUE = "SUN"
BY = "apr.author_name"

BY = "p.clip"
VALUE = "1"

paper_lsit = query_handler.queryPaperBy(BY, VALUE)
print(len(paper_lsit))

11


In [4]:

for paper in paper_lsit :
    doi = paper.DOI

    base_paper = None
    for v in whole_paper_dict.values() :
        if v.DOI == doi :
            base_paper = v
            break
    
    #print(len(paper.reference_list), len(base_paper.reference_list))
    '''
    if paper.authors is not None :
        print(sorted(paper.authors))
    if base_paper.authors is not None :
        print(sorted(base_paper.authors))
    '''
    

    if paper.keywords is not None :
        print("d", sorted(paper.keywords))
    if base_paper.keywords is not None :
        print("b", sorted(base_paper.keywords))


    #print(paper.abstract)
    #print(base_paper.abstract)
    
    #print(paper.title)
    #print(base_paper.title)


    #print(paper.conference_acronym)
    #print(base_paper.conference_acronym)

    #print(sorted(paper.reference_list))
    #print(sorted(base_paper.reference_list))

    #print(paper.DOI)
    #print(base_paper.DOI)
    #print(paper.DOI == base_paper.DOI)

    print("------------")

d ['computer vision', 'machine learning', 'neuroscience']
b ['Computer Vision', 'Machine Learning', 'Neuroscience']
------------
d ['computer vision', 'machine learning', 'medical image analysis']
b ['Computer Vision', 'Machine Learning', 'Medical Image Analysis']
------------
d ['']
------------
d ['']
b ['attention', 'computational neuroscience', 'computer vision', 'robotics', 'vision']
------------
d ['artificial intelligence', 'computer vision', 'machine learning', 'multimedia', 'robotics']
b ['Artificial Intelligence', 'Computer Vision', 'Machine Learning', 'Multimedia', 'Robotics']
------------
d ['']
------------
d ['']
------------
d ['computer vision', 'deep learning', 'image processing', 'machine learning', 'multimedia']
b ['computer vision', 'deep learning', 'image processing', 'machine learning', 'multimedia']
------------
d ['']
------------
d ['artificial intelligence', 'computer vision', 'deep learning', 'machine learning']
b ['Computer vision', 'deep learning', 'machine

In [34]:
for paper in paper_lsit :
    paper.is_in_favorite = False
    query_handler.updatePaper(paper)

In [6]:
query_handler.fillReferenceList(paper)

In [7]:
for doi in paper.reference_list :
    print(query_handler.paperByDOI(doi))

(False, Paper(DOI='10.1016/j.jss.2010.04.065', crossref_json=None, google_schorlar_metadata=None, title='10.1016/j.jss.2010.04.065', authors=None, abstract=None, conference=None, journal=None, year=None, reference_list=None, referenced_list=None, referenced_num=None, cite_bibtex=None, issn_type=None, url=None, is_in_favorite=False, keywords=None, conference_acronym=None, publisher=None, query_handler=None))
(False, Paper(DOI='10.1109/MIS.2012.113', crossref_json=None, google_schorlar_metadata=None, title='10.1109/MIS.2012.113', authors=None, abstract=None, conference=None, journal=None, year=None, reference_list=None, referenced_list=None, referenced_num=None, cite_bibtex=None, issn_type=None, url=None, is_in_favorite=False, keywords=None, conference_acronym=None, publisher=None, query_handler=None))
(False, Paper(DOI='10.1016/j.neucom.2015.07.131', crossref_json=None, google_schorlar_metadata=None, title='10.1016/j.neucom.2015.07.131', authors=None, abstract=None, conference=None, jou

In [82]:
for doi in paper.reference_list :
    print(doi)
    query_handler.queryPaperBy("p.DOI", doi)

10.1016/j.jss.2010.04.065
10.1109/MIS.2012.113
10.1016/j.neucom.2015.07.131
10.1049/iet-ipr.2016.0193
10.3390/s150202369
10.1049/iet-ipr.2014.0935
10.1109/TIM.2011.2175833
10.1016/j.jvcir.2006.12.003
10.1109/TCSVT.2011.2157190
10.1109/TCSVT.2015.2392531
10.1109/TIP.2013.2258353
10.1016/j.patrec.2005.06.015
10.1109/TMECH.2007.897260
10.1109/TCSVT.2010.2045813
10.3390/s140917112
10.1016/j.firesaf.2008.07.006
10.1109/TCSVT.2014.2339592
10.1016/j.firesaf.2008.05.005
10.1016/j.firesaf.2006.02.001
10.1016/S0167-8655(01)00135-0
10.1109/TCSVT.2009.2017419
10.1109/TIP.2015.2475625
10.1016/j.jvcir.2017.02.011
10.1109/TII.2017.2657545
10.1109/TPAMI.2015.2437384
10.1016/j.neuroimage.2014.12.061
10.1016/j.neucom.2016.12.038
10.1016/j.neucom.2016.10.049
10.1109/TKDE.2009.191
10.1016/j.adhoc.2011.01.021
10.1007/s11277-015-2961-6
10.1016/j.comnet.2006.10.002
10.1016/j.phycom.2012.07.005
10.1016/j.inffus.2014.07.002
10.1016/j.patrec.2012.07.018
10.1007/s00138-011-0369-1
10.1016/j.bspc.2016.11.011
10.11

In [52]:
for paper in paper_lsit :
    paper.is_in_favorite = 1
    query_handler.updatePaper(paper)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [48]:
succeed, paper = query_handler.paperByDOI("10.1016/0010-0277(83)90026-4")

In [50]:
pprint(paper)

Paper(DOI='10.1016/0010-0277(83)90026-4',
      crossref_json=None,
      google_schorlar_metadata=None,
      title='10.1016/0010-0277(83)90026-4',
      authors=None,
      abstract=None,
      conference=None,
      journal=None,
      year=None,
      reference_list=None,
      referenced_list=None,
      referenced_num=None,
      cite_bibtex=None,
      issn_type=None,
      url=None,
      is_in_favorite=False,
      keywords=None,
      conference_acronym=None,
      publisher=None,
      query_handler=None)
